In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = '/content/drive/MyDrive/Voice_Data/KoG2P/'  

In [ ]:
! pip install jamotools
! pip install g2pk

In [4]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook
from g2pk import G2p
g2p = G2p()
import jamotools
import time

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [5]:
os.listdir(path)

['aihub_txt.ipynb',
 'broadcast_01_scripts.txt',
 'hobby_01_scripts.txt',
 'dialog_01_scripts.txt',
 'weather_01_scripts.txt',
 'economy_01_scripts.txt',
 'play_01_scripts.txt',
 'shopping_01_scripts.txt',
 'val_shopping_01_scripts.txt',
 'life_01_scripts.txt',
 '.git',
 'LICENSE.md',
 'README.md',
 'g2p.py',
 'rulebook.txt',
 'testset.txt',
 '__pycache__',
 'Text2Phoneme.ipynb',
 'txt_dataframe',
 'val']

In [6]:
import pandas as pd

kor_eng_list=[
['ㅂ','p0'],
['ㅍ','ph'],
['ㅃ','pp'],
['ㄷ','t0'],
['ㅌ','th'],
['ㄸ','tt'],
['ㄱ','k0'],
['ㅋ','kh'],
['ㄲ','kk'],
['ㅅ','s0'],
['ㅆ','ss'],
['ㅎ','h0'],
['ㅈ','c0'],
['ㅊ','ch'],
['ㅉ','cc'],
['ㅁ','mm'],
['ㄴ','nn'],
['ㄹ','rr'],
['ㅂ','pf'],
['ㅍ','ph'],
['ㄷ','tf'],
['ㅌ','th'],
['ㄱ','kf'],
['ㅋ','kh'],
['ㄲ','kk'],
['ㅅ','s0'],
['ㅆ','ss'],
['ㅎ','h0'],
['ㅈ','c0'],
['ㅊ','ch'],
['ㅁ','mf'],
['ㄴ','nf'],
['ㅇ','ng'],
['ㄹ','ll'],
['ㄱㅅ','ks'],
['ㄴㅈ','nc'],
['ㄴㅎ','nh'],
['ㄹㄱ','lk'],
['ㄹㅁ','lm'],
['ㄹㅂ','lb'],
['ㄹㅅ','ls'],
['ㄹㅌ','lt'],
['ㄹㅍ','lp'],
['ㄹㅎ','lh'],
['ㅂㅅ','ps'],
['ㅣ','ii'],
['ㅔ','ee'],
['ㅐ','qq'],
['ㅏ','aa'],
['ㅡ','xx'],
['ㅓ','vv'],
['ㅜ','uu'],
['ㅗ','oo'],
['ㅖ','ye'],
['ㅒ','yq'],
['ㅑ','ya'],
['ㅕ','yv'],
['ㅠ','yu'],
['ㅛ','yo'],
['ㅟ','wi'],
['ㅚ','wo'],
['ㅙ','wq'],
['ㅞ','we'],
['ㅘ','wa'],
['ㅝ','wv'],
['ㅢ','xi']]

eng_kor_list = [[i[1], i[0]] for i in kor_eng_list]
df = pd.DataFrame(eng_kor_list, columns=['eng', 'kor'])

def text_phoneme(text):

    text_eng_phoneme = ! python3 g2p.py f"{text}"

    kor_phoneme = [df.loc[df.eng == k].kor.values[0] for i in text_eng_phoneme[:] for k in i.split()]

    return kor_phoneme

In [7]:
def mkdf(path):
    df = pd.read_csv(path, delimiter=' :: ', header=None) 
    df.columns=['path', 'txt']
    return df 

def bracket_filter(sentence):
    new_sentence = str()
    flag = False
    
    for ch in sentence:
        if ch == '(' and flag == False: 
            flag = True
            continue
        if ch == '(' and flag == True:
            flag = False
            continue
        if ch != ')' and flag == False:
            new_sentence += ch
    return new_sentence

def special_filter(sentence):
    
    try:
        import re
    except ImportError:
        print("pip install re")
        
    SENTENCE_MARK = ['?', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', '.', ',']
    
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            # o/, n/ 등 처리
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
                continue 

        if ch == '#': 
            new_sentence += '샾'

        elif ch not in EXCEPT: 
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence

def sentence_filter(raw_sentence):
    return special_filter(bracket_filter(raw_sentence))
    

In [8]:
from glob import glob
txt = glob(path+'*.txt')[:-2]
txt

['/content/drive/MyDrive/Voice_Data/KoG2P/broadcast_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/hobby_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/dialog_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/weather_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/economy_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/play_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/shopping_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/val_shopping_01_scripts.txt',
 '/content/drive/MyDrive/Voice_Data/KoG2P/life_01_scripts.txt']

In [ ]:
df_broadcast = mkdf(txt[0])
df_hobby = mkdf(txt[1])
df_dialog = mkdf(txt[2])
df_weather = mkdf(txt[3])
df_economy = mkdf(txt[4])
df_play = mkdf(txt[5])
df_shopping = mkdf(txt[6])
df_life = pd.read_csv(path+'life_01_scripts.txt', delimiter='::', header=None) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
# all_df = pd.concat([df_broadcast,df_hobby, df_dialog, df_life, df_weather, df_economy, df_play, df_shopping]).reset_index(drop=True)
all_df = pd.read_csv(path+'txt_dataframe/train_raw_data.csv')
all_df

,path,txt
0,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,(애덤)/(에덤) 스미스의 신념이었죠 그 신념에 대해서 반박하는 이 보기의 사례를 ...
1,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,(애덤)/(에덤) 스미스의 신념이었죠 그 신념에 대해서 반박하는 이 보기의 사례를 ...
2,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,협동을 더 중요하게 봤다.
3,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,우열반 폐지 등수를 없애고 뒤처지는 학생을 끌어올리는 데 집중한 결과 학업성취도 편...
4,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,협동이라는 것을 강조했더니 학업성취도가 높아졌다.
...,...,...
783645,/1.Training/2.원천데이터/8.쇼핑/shopping_01/100/shopp...,아/ 어릴 적부터 친했던 친구분들이 보고 싶으세요?
783646,/1.Training/2.원천데이터/8.쇼핑/shopping_01/100/shopp...,그/ 혹시 집에서 애완동물 키우시나요?
783647,/1.Training/2.원천데이터/8.쇼핑/shopping_01/100/shopp...,음/ 맛있는 음식은 신선한 재료가 생명이지.
783648,/1.Training/2.원천데이터/8.쇼핑/shopping_01/100/shopp...,아/ 다이어트 시작하고 지금까지 치킨 한 번도 안 먹었어요.


In [ ]:
clean_txt = pd.read_csv(path+'txt_dataframe/clean_txt_final.csv') 
clean_txt

,path,txt
0,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,에덤 스미스의 신념이었죠 그 신념에 대해서 반박하는 이 보기의 사례를 찾을 수 있어...
1,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,에덤 스미스의 신념이었죠 그 신념에 대해서 반박하는 이 보기의 사례를 찾을 수 있어...
2,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,협동을 더 중요하게 봤다
3,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,우열반 폐지 등수를 없애고 뒤처지는 학생을 끌어올리는 데 집중한 결과 학업성취도 편...
4,/1.Training/2.원천데이터/1.방송/broadcast_01/001/broa...,협동이라는 것을 강조했더니 학업성취도가 높아졌다
...,...,...
783491,/1.Training/2.원천데이터/8.쇼핑/shopping_01/100/shopp...,아 어릴 적부터 친했던 친구분들이 보고 싶으세요?
783492,/1.Training/2.원천데이터/8.쇼핑/shopping_01/100/shopp...,그 혹시 집에서 애완동물 키우시나요?
783493,/1.Training/2.원천데이터/8.쇼핑/shopping_01/100/shopp...,음 맛있는 음식은 신선한 재료가 생명이지
783494,/1.Training/2.원천데이터/8.쇼핑/shopping_01/100/shopp...,아 다이어트 시작하고 지금까지 치킨 한 번도 안 먹었어요


In [ ]:
a = np.random.randint(clean_txt.shape[0])

print(clean_txt.txt[a])
print(g2p(clean_txt.txt[a]))
print(jamotools.split_syllables(u"{}".format(g2p(clean_txt.txt[a]))))

반복은 반복인데 이 안에 뭐도 있다 층
반보근 반보긴데 이 아네 뭐도 읻따 층
ㅂㅏㄴㅂㅗㄱㅡㄴ ㅂㅏㄴㅂㅗㄱㅣㄴㄷㅔ ㅇㅣ ㅇㅏㄴㅔ ㅁㅝㄷㅗ ㅇㅣㄷㄸㅏ ㅊㅡㅇ


In [ ]:
clean_txt.txt = [g2p(i) for i in tqdm_notebook(clean_txt.txt)] 
clean_txt.to_csv(path+'txt_dataframe/train_g2pk.csv', index=False)

In [ ]:
start = time.time()
jamo = [jamotools.split_syllables(u"{}".format(i)) for i in g2pk] 
print(f"time:{time.time()-start:.4f}") 

time:0.0162


In [9]:
df_broadcast = mkdf(txt[0])
df_hobby = mkdf(txt[1])
df_dialog = mkdf(txt[2])
df_weather = mkdf(txt[3])
df_economy = mkdf(txt[4])
df_play = mkdf(txt[5])
df_shopping = mkdf(txt[6])
df_life = pd.read_csv(path+'life_01_scripts.txt', delimiter='::', header=None) 
df_life.columns = ['path', 'txt']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
pd.DataFrame(list(df_broadcast[df_broadcast.txt.isna()].path)).to_csv(path+'txt_dataframe/broadcast_null_path.csv', index=False)
pd.DataFrame(list(df_hobby[df_hobby.txt.isna()].path)).to_csv(path+'txt_dataframe/hobby_null_path.csv', index=False)
pd.DataFrame(list(df_dialog[df_dialog.txt.isna()].path)).to_csv(path+'txt_dataframe/dialog_null_path.csv', index=False)
pd.DataFrame(list(df_weather[df_weather.txt.isna()].path)).to_csv(path+'txt_dataframe/wheater_null_path.csv', index=False)
pd.DataFrame(list(df_economy[df_economy.txt.isna()].path)).to_csv(path+'txt_dataframe/economy_null_path.csv', index=False)
pd.DataFrame(list(df_play[df_play.txt.isna()].path)).to_csv(path+'txt_dataframe/play_null_path.csv', index=False)
pd.DataFrame(list(df_shopping[df_shopping.txt.isna()].path)).to_csv(path+'txt_dataframe/shopping_null_path.csv', index=False)
pd.DataFrame(list(df_life[df_life.txt.isna()].path)).to_csv(path+'txt_dataframe/life_null_path.csv', index=False)

In [10]:
def pipe(df,name):

    null_idx = df[df.txt.isna()].index
    df.drop(null_idx, axis=0, inplace=True)

    df.txt = [sentence_filter(i) for i in df.txt]

    if df.txt.isna().sum() > 0:
        add_null_idx = df[df.txt.isna()].index
        pd.DataFrame(list(df[df.txt.isna()].path)).to_csv(path+'txt_dataframe/{}_null_path2.csv'.format(name), index=False)

        df.drop(add_null_idx, axis=0, inplace=True)


    df.txt = [g2p(i) for i in tqdm_notebook(df.txt)] 
    df.to_csv(path+'txt_dataframe/{}_g2pk.csv'.format(name), index=False)
    
    df.txt = [jamotools.split_syllables(u"{}".format(i)) for i in df.txt]
    df.to_csv(path+'txt_dataframe/{}_jamo.csv'.format(name), index=False)


In [ ]:
pipe(df_broadcast, 'broadcast')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [ ]:
pipe(df_hobby, 'hobby')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [ ]:
pipe(df_dialog, 'dialog')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [ ]:
pipe(df_weather, 'weather')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [41]:
pipe(df_economy, 'economy')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [ ]:
pipe(df_play, 'play')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [ ]:
pipe(df_shopping, 'shopping')

In [ ]:
pipe(df_life, 'life')